In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import delta_sharing
import pandas as pd

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated
from general_functions.datetime_helper import transform_date_to_timestamp_milliseconds


In [3]:
customer = "LILLYDOO"
min_date = "20250901"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [4]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.{account_id}.conversion_probabilities_view_30_outlook_65840f17a47b22a123981a35" #features_view_30_outlook"
df = delta_sharing.load_as_pandas(table_path)#, limit=20000)

In [ ]:
#df.to_parquet("/Users/karolinegriesbach/Documents/Innkeepr/Git/Analysis/T3_Smartbidding_Updates/data/targeting_history/LILLYDOO/with_db_view_67d1b8102eb78f8bcc5e2c57/views_conversions.parquet")

In [6]:
df["anonymousId"].nunique()

In [7]:
df.groupby("date")["sessionId"].nunique().sort_index(ascending=False)

In [8]:
if min_date is None:
    min_date = df["date"].min()
max_date = df["date"].max()
min_date = pd.to_datetime(min_date).strftime("%Y%m%d")
max_date = pd.to_datetime(max_date).strftime("%Y%m%d")
print(f"Data from {pd.to_datetime(min_date)} to {max_date}")

In [9]:
# return conversions
max_date = pd.to_datetime(max_date)+pd.Timedelta(days=1)
max_date = max_date.strftime("%Y%m%d")
print(f"Getting conversions from {min_date} to {max_date}")
content = {
     "created": {
                     "$gte": transform_date_to_timestamp_milliseconds(min_date),
                     "$lte": transform_date_to_timestamp_milliseconds(max_date),
               }
    }
conversions = send_to_innkeepr_api_paginated(
    f"{url}/conversions/query",
    account_id,
    content,
    logging
    
)

conversions = pd.json_normalize(conversions)
conversions

In [10]:
conversions["date"] = pd.to_datetime(conversions["created"]).dt.date
conversions.groupby("date")["sessionId"].nunique().sort_index(ascending=False)

In [11]:
 # merge conversions and sessions
sessions_with_conversions = df[["date","created","sessionId","anonymousId"]]
sessions_with_conversions = pd.merge(sessions_with_conversions, conversions[conversions["name"]=="checkout_completed"][["sessionId","created","name"]], how="left", on="sessionId", suffixes=("", "_conv"))
sessions_with_conversions

In [13]:
sessions_with_conversions.groupby(by=["date"])["name"].value_counts()

In [15]:
conv_sessions = sessions_with_conversions[sessions_with_conversions["name"]=='checkout_completed']["sessionId"].dropna().unique().tolist()
len(conv_sessions)

In [16]:
sesssions_query = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"sessionId":conv_sessions},
    logging
)

In [17]:
sessions_query_pd = pd.json_normalize(sesssions_query)
sessions_query_pd

In [19]:
sessions_query_pd["date"] = pd.to_datetime(sessions_query_pd["created"]).dt.date
sessions_query_pd.groupby("date")["campaign.gclid"].nunique()

In [20]:
sessions_query_pd_gclid = sessions_query_pd[sessions_query_pd["campaign.gclid"].notna()]
sessions_query_pd_gclid.shape

In [43]:
#fine: 
cate = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/innkeepr-analytics/cate_before_vbt.csv")
#: missing (count 3): cate = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/innkeepr-analytics/before_cate.csv")
cate = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/innkeepr-analytics/cate_after_vbt.csv")

In [44]:
sessions_query_pd_gclid["sessionId"].isin(cate["session"]).sum()

In [38]:
cate.columns

In [39]:
cate

In [41]:
sessions_query_pd_gclid["sessionId"].isin(cate["session"]).sum()

In [42]:
cate2=cate.drop_duplicates(subset=["session"],keep="last")
sessions_query_pd_gclid["sessionId"].isin(cate2["session"]).sum()